In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
if 'NOTEBOOK_INITIATED_FLAG' not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..
import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace
import lib.datagen as datagen

from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf
from tensorflow import keras
import numpy as np

model_name = "pos_batch_deconvolve"
model = keras.models.load_model(f'trained_networks/{model_name}.h5')
#img = image.imread("example_image.png").sum(axis=2)
inputs, img_syn = datagen.get_synthetic_data_pair_positional(randconfig=True, spread=0.01)
distorted_img_syn, xdiff, ydiff = inputs
distorted_img_9 = np.load('example_files/test_data_9.npy')/1e6
distorted_img_0 = np.load('example_files/test_data_0.npy')/1e6


xdiff_r, ydiff_r = xtrace.depth_offsets(datagen.global_config, *utils.ray_grid(datagen.global_config["dimensions"]))
recovered_img_syn = model(np.transpose(inputs, (1, 2, 0))[np.newaxis], training=False).numpy()[0,:,:,0]
recovered_img_9 = model(np.stack((distorted_img_9, xdiff_r, ydiff_r), axis=2)[np.newaxis], training=False).numpy()[0,:,:,0]
recovered_img_0 =  model(np.stack((distorted_img_0, xdiff_r, ydiff_r), axis=2)[np.newaxis], training=False).numpy()[0,:,:,0]

percent = 99.6
perc = np.percentile(distorted_img_0, percent)

fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
axs[0].imshow(img_syn, vmax=perc, vmin=0)
axs[0].set_title("Synthetic Ground Thruth")
axs[1].imshow(distorted_img_syn, vmax=perc, vmin=0)
axs[1].set_title("Synthetic Data")
axs[2].imshow(recovered_img_syn, vmax=perc, vmin=0)
axs[2].set_title("Recovered Synthetic Data")

fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)
axs[0, 0].imshow(distorted_img_0, vmax=perc, vmin=0)
axs[0, 0].set_title("Blurred Captures")
axs[0, 1].imshow(recovered_img_0, vmax=perc, vmin=0)
axs[0, 1].set_title("Recovered DeepGyro Images")
axs[1, 0].imshow(distorted_img_9, vmax=perc, vmin=0)
axs[1, 1].imshow(recovered_img_9, vmax=perc, vmin=0)

plt.show()

G = xtrace.depth_spill_psf(datagen.global_config, *utils.ray_grid(datagen.global_config["dimensions"]))
recovered_img_conv_0 = deconvolution.richard_lucy(distorted_img_0, G, 6)

fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
axs[0].imshow(distorted_img_0, vmax=perc, vmin=0)
axs[0].set_title("Detector Image")
axs[1].imshow(recovered_img_conv_0, vmax=perc, vmin=0)
axs[1].set_title("Richard Lucy Deconvolution")
axs[2].imshow(recovered_img_0, vmax=perc, vmin=0)
axs[2].set_title("ML deconvolution")
plt.show()
# Add local energy loss constraint? (ex: 10x10 area should contain about the same ammount of energy)
# Add positivity loss constraint?
# Try the Pecent contraint with lower randomness value -> trains on more realistic schenarios

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …